In [1]:
%%time
import nxs_integrate
#import display_anim
import glob
import os
from IPython.display import clear_output
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as patches
from IPython.display import HTML
import h5_integrate as integ
from IPython.display import clear_output
from moviepy.editor import ImageSequenceClip
%matplotlib widget

CPU times: user 2.04 s, sys: 63.8 ms, total: 2.1 s
Wall time: 601 ms


## Data extraction from h5

The test data correspond to Co13_2_500. 
We will integrate using sectors of 20° wide (+ or - 10°)

In [2]:
data_dir='/home-local/ratel-ra/Documents/SAXS_data/SAXS_SWING/20231871/test/feb36_85_ind5'
mask='/home-local/ratel-ra/Documents/SAXS_data/SAXS_SWING/20231871/mask.edf'
dir=integ.extract_h5_from_dir(data_dir,10,maskfile=mask)

      

File 24 out  of 24  files in the directory /home-local/ratel-ra/Documents/SAXS_data/SAXS_SWING/20231871/test/feb36_85_ind5
lacroix_00298_2024-07-19_19-27-16.h5


## Definition of working directories

In [3]:
# to avoid reexectuion of the above cell, we redefine dir
#dir='/home-local/ratel-ra/Documents/SAXS_data/SAXS_SWING/20231871/test/co13_2_500'

## Load Numpy files extracted from h5

In [15]:
image_file_list=glob.glob(dir+'/image/'+'*.npy')
image_file_list=sorted(image_file_list,key=integ.extract_number)
positions_file_list=glob.glob(dir+'/positions/'+'*.npy')
positions_file_list=sorted(positions_file_list,key=integ.extract_number)
intensities_file_list=glob.glob(dir+'/integration/'+'*_integrations.npy')
intensities_file_list=sorted(intensities_file_list,key=integ.extract_number)
q_file=glob.glob(dir+'/integration/'+'*_q.npy')[0]
basler_image_file=glob.glob(dir+'/basler_image/'+'*.npy')[0]

## Data visualization

1 numpy file is 1 line

In [16]:
def read_numpy_from_list(file_list):
    #dimension of a single numpy file corresponding to a file
    size=np.shape(np.load(file_list[0]))
    #print('size',size)
    shape=np.concatenate((np.array([len(file_list)]),size))
    #print('shape',shape)
    image = np.zeros(shape)

    for i, file in  enumerate(file_list):
        line = np.load(file)
        #print(i,line)
        image[i] = line
    return image

In [17]:
image = read_numpy_from_list(image_file_list)
print(image.shape)


(24, 10, 1083, 1035)


In [18]:
fig,ax=plt.subplots()
ax.imshow(image[0][0],cmap='viridis')

In [19]:
positions=read_numpy_from_list(positions_file_list)
#print(positions)
x=positions[:,:,0]
z=positions[:,:,1]


In [20]:
basler_image=np.load(basler_image_file)
fig,ax=plt.subplots()
ax.imshow(basler_image,cmap='gray')

#Origin of mapping in lab coordinates (tx4,tz4)
origin_xz=np.array([np.min(x),np.min(z)],dtype=float)
print('origin_xz',origin_xz)

#Convert in µm
x_microns=x*1000
z_microns=z*1000

#ax.scatter(x_microns,z_microns,marker='+')

x_pixels=x_microns/3.73
z_pixels=z_microns/3.67

x_offset=3582
z_offset=6138

x_pixels_ok=x_pixels-x_offset
z_pixels_ok=z_pixels-z_offset


ax.scatter(x_pixels_ok,z_pixels_ok,s=5,marker='.')
# from the beamline settings, we know the coordinates of the beam
origin_pixel=np.array([186,309],dtype=float)

# # #Calculate x grid in pixel coordinates
# x_steps=x_microns+origin_xz[0]
# x_steps_pixel=x_steps*6.702
# x_pixels=x_steps_pixel-origin_pixel[0]
# print(x_pixels)
# # # Calculate z grid in pixel coordinates
# z_steps=z_microns+origin_xz[1]
#  #print(z_steps)
# z_steps_pixel=z_steps*6.812
# print(z_steps_pixel)
# z_pixels=z_steps_pixel+origin_pixel[1]
# print(z_pixels)
#plt.scatter(origin_pixel[0],origin_pixel[1],marker='*',color='r')
#plt.scatter(x,z,marker='+')

origin_xz [14.0996 23.3496]


In [21]:
integrations=read_numpy_from_list(intensities_file_list)
q_array=np.load(q_file)
print(q_array.shape)
qh=q_array[0]
qv=q_array[1]
q_iso=q_array[2]
print(qh.shape,qv.shape,q_iso.shape)
print(integrations.shape)
fig,ax=plt.subplots()
ax.loglog(qh,integrations[0,0,0,:],label='h')
#ax.loglog(qh,integrations[0][0][0])
ax.loglog(qv,integrations[0,0,1,:],label='v')
ax.loglog(q_iso,integrations[0,0,2,:],label='iso')
ax.legend()

(3, 1000)
(1000,) (1000,) (1000,)
(24, 10, 3, 1000)


In [22]:
fig,ax=plt.subplots(1,3,figsize=(15,5))
ax[0].imshow(image[0][0],cmap='viridis')

ax[1].imshow(basler_image,cmap='gray')
ax[1].scatter(x_pixels_ok,z_pixels_ok,s=5,marker='.',color='b')
ax[1].scatter(x_pixels_ok[0][0],z_pixels_ok[0][0],s=10,marker='*',color='r')

ax[2].loglog(qh,integrations[0,0,0,:],label='h')
#ax.loglog(qh,integrations[0][0][0])
ax[2].loglog(qv,integrations[0,0,1,:],label='v')
ax[2].loglog(q_iso,integrations[0,0,2,:],label='iso')
ax[2].legend()
plt.tight_layout()


In [23]:
samplename=data_dir.split('/')[-1]

nbre_lignes=integrations.shape[0]
nbre_colonnes=integrations.shape[1]
plotdir=dir+'/plots'
os.makedirs(plotdir,exist_ok=True)
plt.ioff()
k=1
nbre_fichiers=nbre_colonnes*nbre_lignes
for i in range (nbre_lignes):
    for j in range(nbre_colonnes):
        fig,ax=plt.subplots(1,3,figsize=(15,5))
        ax[0].imshow(image[i][j],cmap='viridis')

        ax[1].imshow(basler_image,cmap='gray')
        ax[1].scatter(x_pixels_ok,z_pixels_ok,s=5,marker='.',color='b')
        ax[1].scatter(x_pixels_ok[i][j],z_pixels_ok[i][j],s=10,marker='*',color='r')

        ax[2].loglog(qh,integrations[i,j,0,:],label='h')
        #ax.loglog(qh,integrations[0][0][0])
        ax[2].loglog(qv,integrations[i,j,1,:],label='v')
        ax[2].loglog(q_iso,integrations[i,j,2,:],label='iso')
        ax[2].legend()
        fig.suptitle(samplename+'x=%.2f'%x[i,j]+', z=%.2f'%z[i,j])
        figname=plotdir+'/plot_line%02d'%i+'_column%02d'%j+'.png'
        
        plt.tight_layout()
        plt.savefig(figname)
        print('File %d'%k, ' out of %d '%nbre_fichiers)
        clear_output(wait=True)
        k+=1
      
        


File 240  out of 240 


In [24]:
 #Specify the path to your images
image_folder = plotdir
plotfile_list=[]
for i in range(nbre_lignes):
    for j in range (nbre_colonnes):
        plotfile_list.append('plot_line%02d'%i+'_column%02d'%j+'.png')

image_files = [f'{image_folder}/{file}' for file in plotfile_list]  # Adjust the range and naming as needed

# Create a video clip from the images
clip = ImageSequenceClip(image_files, fps=12)  # Set the desired frames per second (fps)

# Write the video file
clip.write_videofile(plotdir+'/'+f'{samplename}_video.mp4', codec='libx264')  # Output video file

Moviepy - Building video /home-local/ratel-ra/Documents/SAXS_data/SAXS_SWING/20231871/test/feb36_85_ind5/Feb36_85_ind5/plots/feb36_85_ind5_video.mp4.
Moviepy - Writing video /home-local/ratel-ra/Documents/SAXS_data/SAXS_SWING/20231871/test/feb36_85_ind5/Feb36_85_ind5/plots/feb36_85_ind5_video.mp4



Moviepy - Done !
Moviepy - video ready /home-local/ratel-ra/Documents/SAXS_data/SAXS_SWING/20231871/test/feb36_85_ind5/Feb36_85_ind5/plots/feb36_85_ind5_video.mp4
